# WBGT Analysis for Ironman Triathlons

This notebook analyzes Wet Bulb Globe Temperature (WBGT) values in Ironman and 70.3 races to validate if high values (≥40°C) are reasonable given the race regulations.

## Ironman WBGT Regulations

According to Ironman and World Triathlon guidelines, WBGT thresholds for race modification/cancellation:

| Flag Color | Risk Level | WBGT Range (°C) | Action |
|------------|------------|-----------------|--------|
| Green | Low | < 28.0 | Normal operations |
| Yellow | Moderate | 28.0 - 29.9 | Increased precautions |
| Red | High | 30.0 - 32.0 | Additional precautions or modifications |
| Black | Extreme | > 32.0 | Event modification or cancellation |

Racing is typically modified or cancelled when WBGT exceeds 32.0°C. Values of 40°C+ are extremely rare and would indicate severe heat stress conditions in which no competition would occur.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df1 = pd.read_csv('S6_70.3_weather.csv')
df2 = pd.read_csv('S6_Ironman_weather.csv')

In [ ]:
sns.histplot(df1['WBGT'], bins=20, edgecolor='black')

In [ ]:
sns.histplot(df2['WBGT'], bins=20, edgecolor='black')

In [ ]:
# Define WBGT thresholds according to Ironman regulations
wbgt_thresholds = {
    "Green (Low Risk)": 28.0,
    "Yellow (Moderate Risk)": 30.0,
    "Red (High Risk)": 32.0,
    "Black (Extreme Risk)": 32.1  # Race may be modified or cancelled above this
}

# Statistical summary of WBGT for both datasets
print("WBGT Summary Statistics for 70.3 Races:")
display(df1['WBGT'].describe())

print("\nWBGT Summary Statistics for Full Ironman Races:")
display(df2['WBGT'].describe())

# Check for potentially incorrect high values (WBGT > 40°C)
print(f"\nNumber of 70.3 races with WBGT > 40°C: {(df1['WBGT'] > 40).sum()}")
print(f"Number of Full Ironman races with WBGT > 40°C: {(df2['WBGT'] > 40).sum()}")

# If there are values above 40°C, display those records
if (df1['WBGT'] > 40).sum() > 0:
    print("\nRecords with WBGT > 40°C in 70.3 races:")
    display(df1[df1['WBGT'] > 40][['Date', 'Location', 'Race', 'max_temperature', 'relative_humidity', 'solar_radiation', 'WBGT']])

if (df2['WBGT'] > 40).sum() > 0:
    print("\nRecords with WBGT > 40°C in Full Ironman races:")
    display(df2[df2['WBGT'] > 40][['Date', 'Location', 'Race', 'max_temperature', 'relative_humidity', 'solar_radiation', 'WBGT']])

In [ ]:
# Create plots with regulation thresholds
plt.figure(figsize=(14, 6))

# Plot for 70.3 races
plt.subplot(1, 2, 1)
sns.histplot(df1['WBGT'], bins=30, kde=True, edgecolor='black')

# Add vertical lines for thresholds
for risk, value in wbgt_thresholds.items():
    if risk == "Green (Low Risk)":
        plt.axvline(x=value, color='green', linestyle='--', label=f'{risk}: {value}°C')
    elif risk == "Yellow (Moderate Risk)":
        plt.axvline(x=value, color='yellow', linestyle='--', label=f'{risk}: {value}°C')
    elif risk == "Red (High Risk)":
        plt.axvline(x=value, color='red', linestyle='--', label=f'{risk}: {value}°C')
    else:
        plt.axvline(x=value, color='black', linestyle='--', label=f'{risk}: >{value}°C')

plt.title('WBGT Distribution for 70.3 Races with Risk Thresholds')
plt.xlabel('WBGT (°C)')
plt.ylabel('Frequency')
plt.legend()

# Plot for Full Ironman races
plt.subplot(1, 2, 2)
sns.histplot(df2['WBGT'], bins=30, kde=True, edgecolor='black')

# Add vertical lines for thresholds
for risk, value in wbgt_thresholds.items():
    if risk == "Green (Low Risk)":
        plt.axvline(x=value, color='green', linestyle='--', label=f'{risk}: {value}°C')
    elif risk == "Yellow (Moderate Risk)":
        plt.axvline(x=value, color='yellow', linestyle='--', label=f'{risk}: {value}°C')
    elif risk == "Red (High Risk)":
        plt.axvline(x=value, color='red', linestyle='--', label=f'{risk}: {value}°C')
    else:
        plt.axvline(x=value, color='black', linestyle='--', label=f'{risk}: >{value}°C')

plt.title('WBGT Distribution for Full Ironman Races with Risk Thresholds')
plt.xlabel('WBGT (°C)')
plt.ylabel('Frequency')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Analyze the relationship between temperature, humidity and WBGT
import numpy as np
plt.figure(figsize=(14, 6))

# Scatter plot for 70.3 races
plt.subplot(1, 2, 1)
scatter = plt.scatter(df1['max_temperature'], df1['WBGT'], 
                     c=df1['relative_humidity'], cmap='viridis', 
                     alpha=0.7, s=50)
plt.colorbar(scatter, label='Relative Humidity (%)')

# Add identity line (WBGT=Temperature) for reference
temp_range = np.linspace(df1['max_temperature'].min(), df1['max_temperature'].max(), 100)
plt.plot(temp_range, temp_range, 'r--', alpha=0.7, label='WBGT = Air Temp')

plt.title('WBGT vs. Temperature (70.3 Races)')
plt.xlabel('Maximum Temperature (°C)')
plt.ylabel('WBGT (°C)')
plt.legend()

# Draw horizontal lines for regulation thresholds
for risk, value in wbgt_thresholds.items():
    if risk != "Black (Extreme Risk)":  # Skip black to reduce clutter
        if risk == "Green (Low Risk)":
            plt.axhline(y=value, color='green', linestyle=':', alpha=0.7)
        elif risk == "Yellow (Moderate Risk)":
            plt.axhline(y=value, color='orange', linestyle=':', alpha=0.7)
        elif risk == "Red (High Risk)":
            plt.axhline(y=value, color='red', linestyle=':', alpha=0.7)

# Scatter plot for Full Ironman races
plt.subplot(1, 2, 2)
scatter = plt.scatter(df2['max_temperature'], df2['WBGT'], 
                     c=df2['relative_humidity'], cmap='viridis', 
                     alpha=0.7, s=50)
plt.colorbar(scatter, label='Relative Humidity (%)')

# Add identity line (WBGT=Temperature) for reference
temp_range = np.linspace(df2['max_temperature'].min(), df2['max_temperature'].max(), 100)
plt.plot(temp_range, temp_range, 'r--', alpha=0.7, label='WBGT = Air Temp')

plt.title('WBGT vs. Temperature (Full Ironman Races)')
plt.xlabel('Maximum Temperature (°C)')
plt.ylabel('WBGT (°C)')
plt.legend()

# Draw horizontal lines for regulation thresholds
for risk, value in wbgt_thresholds.items():
    if risk != "Black (Extreme Risk)":  # Skip black to reduce clutter
        if risk == "Green (Low Risk)":
            plt.axhline(y=value, color='green', linestyle=':', alpha=0.7)
        elif risk == "Yellow (Moderate Risk)":
            plt.axhline(y=value, color='orange', linestyle=':', alpha=0.7)
        elif risk == "Red (High Risk)":
            plt.axhline(y=value, color='red', linestyle=':', alpha=0.7)

plt.tight_layout()
plt.show()

In [ ]:
# Simplified WBGT calculation for validation purposes
import numpy as np

def simplified_wbgt(Ta, RH, wind, solar):
    """Simplified WBGT estimate based on air temperature and humidity
    
    This is not as accurate as the full calculation but helps catch gross errors
    """
    # Stull formula for wet-bulb temperature (simpler approximation)
    Tw = Ta * np.arctan(0.151977 * np.sqrt(RH + 8.313659)) + np.arctan(Ta + RH) - np.arctan(RH - 1.676331) + 0.00391838 * (RH**1.5) * np.arctan(0.023101 * RH) - 4.686035
    
    # Simple approximation of natural wet-bulb temperature effect
    # WBGT ≈ 0.7 × Tw + 0.2 × Tg + 0.1 × Ta
    # Approximating Tg as Ta + 2-8°C depending on solar radiation
    solar_factor = np.minimum(8, solar / 150)  # Caps at 8°C for high solar radiation
    Tg = Ta + solar_factor
    
    return 0.7 * Tw + 0.2 * Tg + 0.1 * Ta

# Define a function to validate if WBGT values are reasonable
def validate_wbgt(row):
    """Check if the WBGT value is reasonable based on physics principles"""
    
    # In general, WBGT is usually less than or equal to air temperature
    # unless there's very high humidity AND high solar radiation
    
    # Factors that could lead to suspiciously high WBGT:
    # 1. WBGT significantly higher than air temperature (>10°C difference)
    # 2. WBGT > 40°C when air temperature < 35°C
    
    wbgt = row['WBGT']
    temp = row['max_temperature']
    humidity = row['relative_humidity']
    solar = row['solar_radiation']
    wind = row['average_wind_speed']
    
    issues = []
    
    # Check 1: WBGT should rarely be more than 8°C above air temperature
    # (This can happen with very high humidity and solar radiation)
    if wbgt > temp + 8:
        issues.append(f"WBGT ({wbgt:.1f}°C) is more than 8°C above air temperature ({temp:.1f}°C)")
    
    # Check 2: WBGT > 40°C is extremely rare
    if wbgt > 40:
        issues.append(f"WBGT value ({wbgt:.1f}°C) is extremely high (>40°C)")
        
        # If WBGT > 40 but temperature < 35, this is especially suspicious
        if temp < 35:
            issues.append(f"WBGT > 40°C with temperature < 35°C is physically questionable")
    
    # Check 3: Use simplified WBGT calculation for comparison
    try:
        estimated_wbgt = simplified_wbgt(temp, humidity, wind, solar)
        
        # Check if there's a significant difference (>5°C)
        if abs(estimated_wbgt - wbgt) > 5:
            issues.append(f"Large difference between stored WBGT ({wbgt:.1f}°C) and estimated value ({estimated_wbgt:.1f}°C)")
    except Exception as e:
        issues.append(f"Error in simplified WBGT calculation: {str(e)}")
    
    return issues

# Apply the validation function to both datasets
print("Validating WBGT values in 70.3 races dataset...")
validation_results_70_3 = df1.apply(validate_wbgt, axis=1)

print("Validating WBGT values in Full Ironman races dataset...")
validation_results_ironman = df2.apply(validate_wbgt, axis=1)

# Display rows with potential issues
print("\nPotential issues in 70.3 races dataset:")
for i, issues in enumerate(validation_results_70_3):
    if issues:
        print(f"\nRow {i} ({df1.iloc[i]['Location']} on {df1.iloc[i]['Date']}):")
        for issue in issues:
            print(f"  - {issue}")
        print(f"  Data: Temp={df1.iloc[i]['max_temperature']}°C, RH={df1.iloc[i]['relative_humidity']}%, " +
              f"Wind={df1.iloc[i]['average_wind_speed']}m/s, Solar={df1.iloc[i]['solar_radiation']}W/m²")

print("\nPotential issues in Full Ironman races dataset:")
for i, issues in enumerate(validation_results_ironman):
    if issues:
        print(f"\nRow {i} ({df2.iloc[i]['Location']} on {df2.iloc[i]['Date']}):")
        for issue in issues:
            print(f"  - {issue}")
        print(f"  Data: Temp={df2.iloc[i]['max_temperature']}°C, RH={df2.iloc[i]['relative_humidity']}%, " +
              f"Wind={df2.iloc[i]['average_wind_speed']}m/s, Solar={df2.iloc[i]['solar_radiation']}W/m²")

In [ ]:
import numpy as np
import pandas as pd

# Method 1: Australian Bureau of Meteorology approach (simplified method)
def compute_WBGT_BoM(Ta, RH, wind, solar):
    """
    Compute WBGT using the Australian Bureau of Meteorology approach.
    This is a validated empirical method that combines Tw and Ta.
    
    Args:
        Ta (float): Air temperature in °C
        RH (float): Relative humidity in % (0-100)
        wind (float): Wind speed in m/s
        solar (float): Solar radiation in W/m²
        
    Returns:
        float: WBGT in °C
    """
    # Calculate natural wet-bulb temperature using Stull formula
    Tw = Ta * np.arctan(0.151977 * np.sqrt(RH + 8.313659)) + \
         np.arctan(Ta + RH) - \
         np.arctan(RH - 1.676331) + \
         0.00391838 * RH**1.5 * np.arctan(0.023101 * RH) - \
         4.686035
    
    # Adjust for solar radiation effect (approximation)
    # Solar factor maxes out at about 8°C for very high radiation (1000 W/m²)
    solar_factor = np.minimum(8, solar / 125)
    
    # For outdoor WBGT with solar load
    if solar > 0:
        # Standard formula: WBGT = 0.7*Tw + 0.2*Tg + 0.1*Ta
        # Approximate Tg as Ta + solar_factor based on empirical observations
        return 0.7 * Tw + 0.2 * (Ta + solar_factor) + 0.1 * Ta
    else:
        # Indoor WBGT (no solar load): WBGT = 0.7*Tw + 0.3*Ta
        return 0.7 * Tw + 0.3 * Ta

# Method 2: Liljegren method - simplified implementation
def compute_WBGT_Liljegren(Ta, RH, wind, solar):
    """
    Compute WBGT using a simplified version of Liljegren et al. (2008) method.
    This method is used by the US military and sports organizations.
    
    Args:
        Ta (float): Air temperature in °C
        RH (float): Relative humidity in % (0-100)
        wind (float): Wind speed in m/s
        solar (float): Solar radiation in W/m²
        
    Returns:
        float: WBGT in °C
    """
    # Constants
    emissivity = 0.95
    absorptivity = 0.7
    
    # Saturation vapor pressure (Bolton formula)
    es = 6.112 * np.exp(17.67 * Ta / (Ta + 243.5))
    
    # Actual vapor pressure
    ea = es * RH / 100.0
    
    # Psychrometric constant
    gamma = 0.000665
    
    # Effective wind speed - cap at minimum of 0.5 m/s for natural convection
    wind_eff = np.maximum(0.5, wind)
    
    # Calculate wet-bulb temperature using psychrometric approximation
    delta = 4098 * es / ((Ta + 237.3)**2)
    Tw = Ta - (ea - es) / (gamma * (1 + delta/gamma))
    
    # Adjusted for ventilation (wind effect on wet bulb)
    if wind_eff > 3.0:
        Tw = Tw - 0.5 * (wind_eff - 3.0) / 3.0
    
    # Approximate black globe temperature with limits
    # Solar radiation effect on globe temperature
    Tg_solar = Ta
    if solar > 0:
        # Radiation factor (simplified from Liljegren)
        rad_factor = absorptivity * solar / (5.67e-8 * emissivity * (Ta+273.15)**4 + 6.5 * wind_eff**0.6)
        Tg_solar = Ta + np.minimum(8.0, rad_factor)  # Cap the increase at 8°C
    
    # Limit extreme values - caps WBGT at physically reasonable values
    Tw = np.minimum(Ta + 2, Tw)  # Wet bulb rarely exceeds dry bulb by more than 2°C
    Tg_solar = np.minimum(Ta + 15, Tg_solar)  # Globe rarely exceeds air temp by more than 15°C
    
    # Calculate WBGT
    WBGT = 0.7 * Tw + 0.2 * Tg_solar + 0.1 * Ta
    
    # Additional safety cap - WBGT should not exceed 40°C except in extreme conditions
    WBGT = np.minimum(40.0, WBGT)
    
    return WBGT

# Method 3: Improved Kong & Huber method with corrections
def compute_WBGT_improved(Ta, RH, wind, solar, P=101325):
    """
    Improved version of the Kong & Huber WBGT calculation with corrected solar treatment
    and better handling of extreme conditions.
    
    Args:
        Ta (float): Air temperature in °C
        RH (float): Relative humidity in % (0-100)
        wind (float): Wind speed in m/s
        solar (float): Solar radiation in W/m²
        P (float): Atmospheric pressure in Pa (default 101325 Pa)
        
    Returns:
        float: WBGT in °C
    """
    # Constants (same as original)
    sigma = 5.670374419e-8    # Stefan-Boltzmann (W/m²/K⁴)
    eps_g = 0.95             # emissivity of black globe
    eps_w = 0.95             # emissivity of wet wick
    D_g = 0.15               # globe diameter (m)
    D_w = 0.025              # wick cylinder diameter (m)
    k_air = 0.026            # thermal conductivity of air (W/m/K)
    cp_air = 1005.0          # specific heat of air at constant pressure (J/kg/K)
    rho_air = 1.225          # air density (kg/m³)
    M_H2O = 0.01801528       # molar mass of water (kg/mol)
    DeltaH = 45000.0         # latent heat of vaporization (J/mol)
    
    # Ensure inputs are arrays
    Ta = np.array(Ta, dtype=float)
    RH = np.array(RH, dtype=float)
    wind = np.array(wind, dtype=float)
    S = np.array(solar, dtype=float)
    
    # Apply limits to input parameters to avoid unrealistic values
    wind = np.maximum(0.1, wind)  # Minimum wind speed of 0.1 m/s for natural convection
    RH = np.minimum(100.0, np.maximum(1.0, RH))  # RH between 1% and 100%
    S = np.minimum(1200.0, np.maximum(0.0, S))  # Limit solar radiation to reasonable values
    
    # Convert Ta to Kelvin for radiative terms
    Ta_K = Ta + 273.15
    
    # Saturation vapor pressure and actual vapor pressure
    esat = 610.78 * np.exp((17.27 * Ta) / (Ta + 237.3))  # Pa
    e_a = esat * (RH / 100.0)
    
    # Estimate wet-bulb temperature using Stull formula
    Tw = (Ta * np.arctan(0.151977 * np.sqrt(RH + 8.313659))
          + np.arctan(Ta + RH)
          - np.arctan(RH - 1.676331)
          + 0.00391838 * RH**1.5 * np.arctan(0.023101 * RH)
          - 4.686035)
    
    # Apply physical constraint to wet-bulb temperature
    Tw = np.minimum(Ta + 1, Tw)  # Wet-bulb should not exceed dry-bulb by more than 1°C in natural conditions
    
    # Air properties
    nu_air = 1.5e-5    # kinematic viscosity (m²/s)
    Pr = 0.71          # Prandtl number of air
    Sc = 0.62          # Schmidt number for water vapor in air
    
    # Convective heat transfer for globe
    Re_g = (wind * D_g) / nu_air
    Nu_g = 2.0 + 0.6 * np.sqrt(np.maximum(Re_g, 1e-3)) * Pr**(1/3)
    h_cg = Nu_g * k_air / D_g
    
    # Convective heat transfer for cylinder (wet wick)
    Re_w = (wind * D_w) / nu_air
    Sh = (0.3 
          + 0.62 * np.sqrt(np.maximum(Re_w, 1e-3)) * Sc**(1/3)
          * (1 + (0.4/Sc)**(2/3))**0.25
          * (1 + (Re_w/2.82e5)**0.625)**0.8)
    
    Dv = 2.5e-5
    kx = Sh * Dv / D_w
    h_cw = rho_air * cp_air * kx * (Sc**(-2/3))
    
    # Radiative coefficients
    h_rg = 4 * sigma * eps_g * Ta_K**3
    h_rw = 4 * sigma * eps_w * Ta_K**3
    
    # Solar absorptions - CORRECTED from original
    # Adjusting solar absorption coefficients based on standard WBGT instrument characteristics
    # Standard black globe absorbs ~0.95 of incident radiation, but only about 1/4 of the total solar flux
    # hits the globe due to its spherical shape
    SRg = 0.15 * S  # Reduced from 0.25 to 0.15 to better match empirical data
    SRw = 0.10 * S  # Reduced from 0.25 to 0.10 for the cylindrical wick
    
    # Apply a cap to solar effects for extremely high radiation values
    SRg = np.minimum(SRg, 150.0)  # This effectively caps the temperature increase due to radiation
    SRw = np.minimum(SRw, 100.0)
    
    # Evaporative transfer coefficient
    beta_b = kx * M_H2O * DeltaH / P
    dT = np.maximum(Ta - Tw, 1e-6)
    h_dew = beta_b * (esat - e_a) / dT
    
    # Calculate globe and natural wet-bulb temperatures
    # Add minimum bounds to denominators to prevent instability
    Tg_hat = Ta + SRg / np.maximum(h_cg + h_rg, 10.0)
    
    numer_Tnw = SRw - beta_b * (esat - e_a)
    denom_Tnw = h_dew + h_cw + h_rw
    Tnw_hat = Ta + numer_Tnw / np.maximum(denom_Tnw, 10.0)
    
    # Apply physically-based limits to Tg_hat and Tnw_hat
    Tg_hat = np.minimum(Ta + 15, Tg_hat)  # Globe temperature rarely exceeds air temp by >15°C
    Tnw_hat = np.minimum(Ta, np.maximum(Tw, Tnw_hat))  # Natural wet-bulb between Tw and Ta
    
    # Combine into WBGT with the standard 0.7/0.2/0.1 weighting
    WBGT = 0.7 * Tnw_hat + 0.2 * Tg_hat + 0.1 * Ta
    
    # Final sanity check - cap WBGT at physically reasonable values
    # WBGT should not exceed ~35°C except in the most extreme conditions
    WBGT = np.minimum(38.0, WBGT)
    
    return WBGT

# Now apply all three methods to our datasets and compare results
print("Applying improved WBGT calculation methods...")

# Recalculate WBGT with all three methods for 70.3 races
df1['WBGT_BoM'] = df1.apply(lambda row: compute_WBGT_BoM(
    Ta=row['max_temperature'],
    RH=row['relative_humidity'],
    wind=row['average_wind_speed'],
    solar=row['solar_radiation']
), axis=1)

df1['WBGT_Liljegren'] = df1.apply(lambda row: compute_WBGT_Liljegren(
    Ta=row['max_temperature'],
    RH=row['relative_humidity'],
    wind=row['average_wind_speed'],
    solar=row['solar_radiation']
), axis=1)

df1['WBGT_improved'] = df1.apply(lambda row: compute_WBGT_improved(
    Ta=row['max_temperature'],
    RH=row['relative_humidity'],
    wind=row['average_wind_speed'],
    solar=row['solar_radiation']
), axis=1)

# Recalculate WBGT with all three methods for Full Ironman races
df2['WBGT_BoM'] = df2.apply(lambda row: compute_WBGT_BoM(
    Ta=row['max_temperature'],
    RH=row['relative_humidity'],
    wind=row['average_wind_speed'],
    solar=row['solar_radiation']
), axis=1)

df2['WBGT_Liljegren'] = df2.apply(lambda row: compute_WBGT_Liljegren(
    Ta=row['max_temperature'],
    RH=row['relative_humidity'],
    wind=row['average_wind_speed'],
    solar=row['solar_radiation']
), axis=1)

df2['WBGT_improved'] = df2.apply(lambda row: compute_WBGT_improved(
    Ta=row['max_temperature'],
    RH=row['relative_humidity'],
    wind=row['average_wind_speed'],
    solar=row['solar_radiation']
), axis=1)

In [ ]:
# Save the updated datasets with corrected WBGT values
# Rename the original WBGT column to preserve it
df1 = df1.rename(columns={'WBGT': 'WBGT_original'})
df2 = df2.rename(columns={'WBGT': 'WBGT_original'})

# Set the improved WBGT as the new standard WBGT
df1['WBGT'] = df1['WBGT_improved']
df2['WBGT'] = df2['WBGT_improved']

# Save updated files
df1.to_csv('S6_70.3_weather_corrected.csv', index=False)
df2.to_csv('S6_Ironman_weather_corrected.csv', index=False)

print("Updated data saved with corrected WBGT values.")
print("- Original 70.3 races WBGT max: {:.2f}°C".format(df1['WBGT_original'].max()))
print("- Corrected 70.3 races WBGT max: {:.2f}°C".format(df1['WBGT'].max()))
print("- Original full Ironman races WBGT max: {:.2f}°C".format(df2['WBGT_original'].max()))
print("- Corrected full Ironman races WBGT max: {:.2f}°C".format(df2['WBGT'].max()))